In [15]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [16]:
def ex_tag(sid, page):
    ### 뉴스 분야(sid)와 페이지(page)를 입력하면 그에 대한 링크들을 리스트로 추출하는 함수 ###
    
    ## 1.
    url = f"https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={sid}"\
    "#&date=%2000:00:00&page={page}"
    html = requests.get(url, headers={"User-Agent": "Mozilla/5.0"\
    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "\
    "Chrome/110.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")
    a_tag = soup.find_all("a")
    
    ## 2.
    tag_lst = []
    for a in a_tag:
        if "href" in a.attrs:  # href가 있는것만 고르는 것
            if (f"sid={sid}" in a["href"]) and ("article" in a["href"]):
                tag_lst.append(a["href"])
                
    return tag_lst

In [17]:
def re_tag(sid):
    ### 특정 분야의 100페이지까지의 뉴스의 링크를 수집하여 중복 제거한 리스트로 변환하는 함수 ###
    ## 여기 바꾸면 돼  예정!!!!
    re_lst = []
    for i in tqdm(range(101,200)):
        lst = ex_tag(sid, i+1)
        re_lst.extend(lst)

    # 중복 제거
    re_set = set(re_lst)
    re_lst = list(re_set)
    
    return re_lst

In [18]:
all_hrefs = {}
sids = [i for i in range(100,106)]  # 분야 리스트
### 분야 바꾸고 싶으면 여기 바꾸면 됨
# 각 분야별로 링크 수집해서 딕셔너리에 저장
for sid in sids:
    sid_data = re_tag(sid)
    all_hrefs[sid] = sid_data

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

In [19]:
def art_crawl(all_hrefs, sid, index):
    """
    sid와 링크 인덱스를 넣으면 기사제목, 날짜, 본문을 크롤링하여 딕셔너리를 출력하는 함수 
    
    Args: 
        all_hrefs(dict): 각 분야별로 100페이지까지 링크를 수집한 딕셔너리 (key: 분야(sid), value: 링크)
        sid(int): 분야 [100: 정치, 101: 경제, 102: 사회, 103: 생활/문화, 104: 세계, 105: IT/과학]
        index(int): 링크의 인덱스
    
    Returns:
        dict: 기사제목, 날짜, 본문이 크롤링된 딕셔너리
    
    """
    art_dic = {}
    
    ## 1.
    title_selector = "#title_area > span"
    date_selector = "#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans"\
    "> div.media_end_head_info_datestamp > div:nth-child(1) > span"
    main_selector = "#dic_area"
    
    url = all_hrefs[sid][index]
    html = requests.get(url, headers = {"User-Agent": "Mozilla/5.0 "\
    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)"\
    "Chrome/110.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")
    
    ## 2.
    # 제목 수집
    title = soup.select(title_selector)
    title_lst = [t.text for t in title]
    title_str = "".join(title_lst)
    
    # 날짜 수집
    date = soup.select(date_selector)
    date_lst = [d.text for d in date]
    date_str = "".join(date_lst)
    
    # 본문 수집
    main = soup.select(main_selector)
    main_lst = []
    for m in main:
        m_text = m.text
        m_text = m_text.strip()
        main_lst.append(m_text)
    main_str = "".join(main_lst)
    
    ## 3.
    art_dic["title"] = title_str
    art_dic["date"] = date_str
    art_dic["main"] = main_str
    
    return art_dic

In [20]:
# 모든 섹션의 데이터 수집 (제목, 날짜, 본문, section, url)
section_lst = [s for s in range(100, 106)]
artdic_lst = []

for section in tqdm(section_lst):
    for i in tqdm(range(len(all_hrefs[section]))):
        art_dic = art_crawl(all_hrefs, section, i)
        art_dic["section"] = section
        art_dic["url"] = all_hrefs[section][i]
        artdic_lst.append(art_dic)

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/440 [00:00<?, ?it/s]

  0%|          | 0/574 [00:00<?, ?it/s]

In [ ]:
import pandas as pd

art_df = pd.DataFrame(artdic_lst)

In [ ]:
art_df.to_csv("article_df.csv")

In [ ]:
art_df.sort_values(by='date', ascending=True)

,title,date,main,section,url
867,"대웅제약, '펙수클루' 중국 품목허가 신청 완료",2023.06.27. 오전 10:00,대웅제약은 자체 개발한 P-CAB(칼륨 경쟁적 위산분비 억제제) 계열 '펙수클루 4...,101,https://n.news.naver.com/mnews/article/215/000...
1070,내일부터 '만 나이'…네이버 계산기로 바로 확인,2023.06.27. 오전 10:00,"네이버, 만 나이 계산기 서비스다음, 인물 정보에서 만 나이만\n\n\n\n네이버 ...",102,https://n.news.naver.com/mnews/article/011/000...
1195,"부산경찰청, 피서기간 해수욕장서 여름 경찰서·파출소 운영",2023.06.27. 오전 10:01,[부산=뉴시스] 하경민 기자 = 부산 해운대·송정 해수욕장 부분 개장을 하루 앞둔 ...,102,https://n.news.naver.com/mnews/article/003/001...
646,"홈플러스, 당당치킨 이어 '당당버거' 출시...""2개에 4,990원""",2023.06.27. 오전 10:01,홈플러스는 외식 물가 안정에 기여하기 위해 오는 29일부터 신메뉴 '당당 순살치킨 ...,101,https://n.news.naver.com/mnews/article/215/000...
1102,"“이게 왜 킬러 문항?”, “물수능이란 뜻?”…킬러 문항 공개에 수험생·학원가 술렁",2023.06.27. 오전 10:02,"교육부 킬러문항 공개국영수 22개, 과탐 4개 선정 기준 두고 수험생·사교육업계 갸...",102,https://n.news.naver.com/mnews/article/016/000...
...,...,...,...,...,...
761,"현대차, '포니의 시간' 전시에 전직 임원 초청",2023.06.28. 오전 9:59,현대자동차가 전직 임원들을 초청해 최근 복원한 '포니 쿠페 콘셉트'를 소개하는 시간...,101,https://n.news.naver.com/mnews/article/277/000...
1024,오늘부터 '만 나이' 적용...예외 대상은?,2023.06.28. 오전 9:59,[오늘의 건강]\n\n\n\n[사진=게티이미지뱅크]오늘(28일)은 남부지방을 중심으...,102,https://n.news.naver.com/mnews/article/296/000...
1283,"與 문성호 ""황의조 폭로女, 스스로 결정해놓고 男에 책임 넘겨""",2023.06.28. 오전 9:59,지난 20일 대전 유성구 대전월드컵경기장에서 열린 하나은행 초청 대한민국과 엘살바도...,102,https://n.news.naver.com/mnews/article/008/000...
344,당정 “예비군 학습권 보장…2학기 시작 전 시행”,2023.06.28. 오전 9:59,국민의힘과 정부가 예비군 훈련에 참석할 때 결석 처리 등 불이익이 없도록 하는 ‘예...,100,https://n.news.naver.com/mnews/article/056/001...
